In [31]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import MCMT
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display

def load_image(image_path):
    image = Image.open(image_path).convert('L')  # Convert image to grayscale
    image = np.array(image)
    image = (image > 128).astype(int)  # Convert to binary image
    return image

def save_image(image, path):
    image = Image.fromarray((image * 255).astype(np.uint8))
    image.save(path)

def encode_image_to_quantum(image):
    n = image.size
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)
    
    for i, pixel in enumerate(image.flatten()):
        if pixel == 1:
            qc.x(qr[i])
    return qc

def apply_custom_dilation_operator(qc, image_size):
    n = image_size[0] * image_size[1]
    qr = qc.qregs[0]
    
    def apply_custom_gate(control_qubits, target_qubit):
        if control_qubits:  # Only apply gate if there are control qubits
            mcmt_gate = MCMT('cx', len(control_qubits), 1)
            qc.append(mcmt_gate, control_qubits + [target_qubit])

    for i in range(image_size[0]):
        for j in range(image_size[1]):
            idx = i * image_size[1] + j
            neighbors = []
            if i > 0:  # Top neighbor
                neighbors.append(qr[idx - image_size[1]])
            if i < image_size[0] - 1:  # Bottom neighbor
                neighbors.append(qr[idx + image_size[1]])
            if j > 0:  # Left neighbor
                neighbors.append(qr[idx - 1])
            if j < image_size[1] - 1:  # Right neighbor
                neighbors.append(qr[idx + 1])
            
            apply_custom_gate(neighbors, qr[idx])
                
    return qc

def decode_quantum_to_image(counts, image_size):
    image = np.zeros(image_size)
    max_count_key = max(counts, key=counts.get)  # Find the most probable outcome
    for i, bit in enumerate(max_count_key[::-1]):
        image[i // image_size[1], i % image_size[1]] = int(bit)
    return image

def process_chunk(chunk, chunk_shape):
    image_size = chunk_shape
    
    qc = encode_image_to_quantum(chunk)
    qc = apply_custom_dilation_operator(qc, image_size)
    
    cr = ClassicalRegister(image_size[0] * image_size[1])
    qc.add_register(cr)
    qc.measure(range(image_size[0] * image_size[1]), range(image_size[0] * image_size[1]))
    
    backend = Aer.get_backend('qasm_simulator')
    t_qc = transpile(qc, backend, optimization_level=3)  # Use highest optimization level
    job = backend.run(t_qc)
    result = job.result()
    counts = result.get_counts()
    
    dilated_chunk = decode_quantum_to_image(counts, image_size)
    return dilated_chunk

def morphological_dilation(image, chunk_size=(4, 4)):
    image_height, image_width = image.shape
    chunk_height, chunk_width = chunk_size
    
    dilated_image = np.zeros_like(image)
    
    for i in range(0, image_height, chunk_height):
        for j in range(0, image_width, chunk_width):
            chunk = image[i:i+chunk_height, j:j+chunk_width]
            chunk_shape = chunk.shape
            
            # If the chunk is smaller than the chunk size, pad it with zeros
            if chunk.shape[0] != chunk_height or chunk.shape[1] != chunk_width:
                chunk = np.pad(chunk, ((0, chunk_height - chunk.shape[0]), (0, chunk_width - chunk.shape[1])), 'constant')
            
            dilated_chunk = process_chunk(chunk, chunk_shape)
            dilated_image[i:i+chunk_shape[0], j:j+chunk_shape[1]] = dilated_chunk[:chunk_shape[0], :chunk_shape[1]]
    
    fig, axs = plt.subplots(1, 2, figsize=(15, 7))
    axs[0].imshow(image, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(dilated_image, cmap='gray')
    axs[1].set_title('Dilated Image')
    plt.show()

    return dilated_image

# Create file upload widget
file_upload = widgets.FileUpload(accept='.png, .jpg, .jpeg', multiple=False)

# Create chunk size widgets
chunk_height_slider = widgets.IntSlider(value=4, min=1, max=32, step=1, description='Chunk Height:')
chunk_width_slider = widgets.IntSlider(value=4, min=1, max=32, step=1, description='Chunk Width:')
chunk_size_box = widgets.HBox([chunk_height_slider, chunk_width_slider])

# Create button to trigger dilation
dilate_button = widgets.Button(description='Dilate Image')

def on_dilate_button_clicked(b):
    if file_upload.value:
        uploaded_file = next(iter(file_upload.value.values()))  # Properly get the file info
        file_content = uploaded_file['content']
        image = Image.open(io.BytesIO(file_content)).convert('L')
        image_np = np.array(image)
        image_np = (image_np > 128).astype(int)  # Convert to binary image
        chunk_size = (chunk_height_slider.value, chunk_width_slider.value)
        dilated_image = morphological_dilation(image_np, chunk_size)
        save_image(dilated_image, 'dilated_image.png')

dilate_button.on_click(on_dilate_button_clicked)

# Display widgets
display(file_upload)
display(chunk_size_box)
display(dilate_button)


FileUpload(value=(), accept='.png, .jpg, .jpeg', description='Upload')

Button(description='Dilate Image', style=ButtonStyle())

AttributeError: 'tuple' object has no attribute 'values'

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import interact, widgets, FileUpload, IntSlider
from IPython.display import display
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import MCMT

def load_image(image_path):
    image = Image.open(image_path).convert('L')  # Convert image to grayscale
    image = np.array(image)
    image = (image > 128).astype(int)  # Convert to binary image
    return image

def save_image(image, path):
    image = Image.fromarray((image * 255).astype(np.uint8))
    image.save(path)

def encode_image_to_quantum(image):
    n = image.size
    qr = QuantumRegister(n)
    qc = QuantumCircuit(qr)
    
    for i, pixel in enumerate(image.flatten()):
        if pixel == 1:
            qc.x(qr[i])
    return qc

def apply_custom_dilation_operator(qc, image_size):
    n = image_size[0] * image_size[1]
    qr = qc.qregs[0]
    
    def apply_custom_gate(control_qubits, target_qubit):
        mcmt_gate = MCMT('cx', len(control_qubits), 1)
        qc.append(mcmt_gate, control_qubits + [target_qubit])

    for i in range(image_size[0]):
        for j in range(image_size[1]):
            idx = i * image_size[1] + j
            neighbors = []
            if i > 0:  # Top neighbor
                neighbors.append(qr[idx - image_size[1]])
            if i < image_size[0] - 1:  # Bottom neighbor
                neighbors.append(qr[idx + image_size[1]])
            if j > 0:  # Left neighbor
                neighbors.append(qr[idx - 1])
            if j < image_size[1] - 1:  # Right neighbor
                neighbors.append(qr[idx + 1])
            
            if neighbors:
                apply_custom_gate(neighbors, qr[idx])
                
    return qc

def decode_quantum_to_image(counts, image_size):
    image = np.zeros(image_size)
    max_count_key = max(counts, key=counts.get)  # Find the most probable outcome
    for i, bit in enumerate(max_count_key[::-1]):
        image[i // image_size[1], i % image_size[1]] = int(bit)
    return image

def process_chunk(chunk, chunk_shape):
    image_size = chunk_shape
    
    qc = encode_image_to_quantum(chunk)
    
    qc = apply_custom_dilation_operator(qc, image_size)
    
    cr = ClassicalRegister(image_size[0] * image_size[1])
    qc.add_register(cr)
    qc.measure(range(image_size[0] * image_size[1]), range(image_size[0] * image_size[1]))
    
    backend = Aer.get_backend('qasm_simulator')
    t_qc = transpile(qc, backend, optimization_level=3)
    job = backend.run(t_qc)
    result = job.result()
    counts = result.get_counts()
    
    dilated_chunk = decode_quantum_to_image(counts, image_size)
    return dilated_chunk

def morphological_dilation(image, chunk_size=(4, 4)):
    image_height, image_width = image.shape
    chunk_height, chunk_width = chunk_size
    
    dilated_image = np.zeros_like(image)
    
    for i in range(0, image_height, chunk_height):
        for j in range(0, image_width, chunk_width):
            chunk = image[i:i+chunk_height, j:j+chunk_width]
            chunk_shape = chunk.shape
            
            if chunk.shape[0] != chunk_height or chunk.shape[1] != chunk_width:
                chunk = np.pad(chunk, ((0, chunk_height - chunk.shape[0]), (0, chunk_width - chunk.shape[1])), 'constant')
            
            dilated_chunk = process_chunk(chunk, chunk_shape)
            dilated_image[i:i+chunk_shape[0], j:j+chunk_shape[1]] = dilated_chunk[:chunk_shape[0], :chunk_shape[1]]
    
    return dilated_image

def display_images(original_image, dilated_image):
    fig, axs = plt.subplots(1, 2, figsize=(15, 7))
    axs[0].imshow(original_image, cmap='gray')
    axs[0].set_title('Original Image')
    axs[1].imshow(dilated_image, cmap='gray')
    axs[1].set_title('Dilated Image')
    plt.show()

def on_image_upload(change):
    uploaded_image_path = tupple(change['new'].keys())[0]
    with open(uploaded_image_path, 'wb') as f:
        f.write(change['new'][uploaded_image_path]['content'])
    
    original_image = load_image(uploaded_image_path)
    dilated_image = morphological_dilation(original_image, chunk_size=(chunk_size_slider.value, chunk_size_slider.value))
    display_images(original_image, dilated_image)

upload_button = FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_image_upload, names='value')

chunk_size_slider = IntSlider(value=4, min=2, max=10, step=1, description='Chunk Size:', continuous_update=False)

display(upload_button, chunk_size_slider)


FileUpload(value=(), accept='image/*', description='Upload')

IntSlider(value=4, continuous_update=False, description='Chunk Size:', max=10, min=2)

AttributeError: 'tuple' object has no attribute 'keys'